<a href="https://colab.research.google.com/github/HayatoYagi/Elliptic-Curves/blob/main/elliptic_curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import math

def divisors(n):
    res = []
    i = 1
    while(i * i < n):
        if n % i == 0:
            res.append(i)
            res.append(n // i)
        i += 1
    if i * i == n:
        res.append(i)
    return res

class EllipticCurve:
    """
    y^2 = f(x) := x^3 + ax^2 + bx + c (a,b,c: 整数)
    """
    def __init__(self, a: int, b: int, c: int):
        self._a = a
        self._b = b
        self._c = c

        # discriminant
        self._D = (-4 * self._a * self._a * self._a * self._c
                    + self._a * self._a * self._b * self._b
                    + 18 * self._a * self._b * self._c
                    - 4 * self._b * self._b * self._b
                    - 27 * self._c * self._c)
        assert(self._D != 0)
    
    def f(self, x):
        return (x * x * x
                + self._a * x * x
                + self._b * x
                + self._c)
        
    def f_prime(self, x):
        return (3 * x * x
                + 2 * self._a * x
                + self._b)

    def torsion_candidates(self):
        """
        有限位数の有理点の候補を返す。
        必要条件しか確認していないことに注意。
        """
        def y_candidates():
            res = [0]
            D = abs(self._D)
            i = 1
            while i * i <= D:
                if(D % (i * i) == 0):
                    res.append(i)
                i += 1
            return res

        def find_x(y):
            res = []
            a = self._a
            b = self._b
            c = self._c - y * y
            if c == 0:
                res.append(0)
                if b == 0:
                    if(a != 0):
                        res.append(-a)
                else:
                    for d in divisors(abs(b)):
                        if(d * d + a * d + b == 0):
                            res.append(d)
                        if(d * d - a * d + b == 0):
                            res.append(-d)
            else:
                for d in divisors(abs(c)):
                    if(d * d * d + a * d * d + b * d + c == 0):
                        res.append(d)
                    if(- d * d * d + a * d * d - b * d + c == 0):
                        res.append(-d)
            return res

        res = []
        for y in y_candidates():
            for x in find_x(y):
                res.append((x,y))
                if(y != 0):
                    res.append((x,-y))
        return res
    
    def double_x(self, x):
        """
        入力された曲線上の整数点のx座標に対し、その点を2倍した点のx座標とそれが整数かどうかを返す。
        """
        numer = (x * x * x * x
                 - 2 * self._b * x * x
                 - 8 * self._c * x
                 + self._b * self._b
                 - 4 * self._a * self._c)
        denom = 4 * self.f(x)
        if(denom == 0):
            return (math.inf, True)
        else:
            return (numer / denom, numer % denom == 0)

    def add(self, p1, p2):
        if math.isinf(p1[0]):
            # todo
            return (-math.inf,-math.inf)
        x1,y1 = p1
        x2,y2 = p2
        if x1 == x2:
            if y1 == y2:
                if y1 == 0:
                    return (math.inf, math.inf)
                else:
                    la = self.f_prime(x1) / 2 / y1
                    nu = y1 - la * x1
                    x3, _ = self.double_x(x1)
                    y3 = la * x3 + nu
                    return (x3, -y3)
            else:
                return (math.inf, math.inf)
        else:
            la = (y2 - y1) / (x2 - x1) # lambda
            nu = y1 - la * x1 # nu
            x3 = la * la - self._a - x1 - x2
            y3 = la * x3 + nu
            return (x3, -y3)

    def scalar_multiple(self, p, n:int):
        assert(n >= 1)
        if(n == 1):
            return p
        q = self.scalar_multiple(p, n // 2)
        q = self.add(q, q)
        if(n % 2 == 1):
            q = self.add(q, p)
        return q

    def order(self, p):
        q = p
        for i in range(1,14):
            if math.isinf(q[0]):
                return i
            elif not ((type(q[0]) is int) or (q[0].is_integer())):
                return math.inf
            q = self.add(q,p)
        return math.inf

    def torsions(self):
        """
        有限位数の有利点とその位数のtupleのリストを返す
        """
        res = []
        for p in self.torsion_candidates():
            ord = self.order(p)
            if not math.isinf(ord):
                res.append((p, ord))
        return res

    def print_equation(self):
        equation_str = "y^2 = x^3"
        if self._a > 0:
            if self._a == 1:
                equation_str += ' + x^2'
            else:
                equation_str += ' + ' + str(self._a) + 'x^2'
        elif self._a < 0:
            if self._a == -1:
                equation_str += ' - x^2'
            else:
                equation_str += ' - ' + str(abs(self._a)) + 'x^2'
        if self._b > 0:
            if self._b == 1:
                equation_str += ' + x'
            else:
                equation_str += ' + ' + str(self._b) + 'x'
        elif self._b < 0:
            if self._b == -1:
                equation_str += ' - x'
            else:
                equation_str += ' - ' + str(abs(self._b)) + 'x'
        if self._c > 0:
            equation_str += ' + ' + str(self._c)
        elif self._c < 0:
            equation_str += ' - ' + str(abs(self._c))
        print(equation_str)

    def print_info(self):
        print('-' * 50)
        self.print_equation()
        print()

        print('discriminant:', self._D)
        print()

        print('Torsion points: ')
        for p,ord in self.torsions():
            print(p, '位数:', ord)
        print('-' * 50)

In [52]:
e1 = EllipticCurve(1009,-72240,705600)
e1.print_info()

e1.p1 = (0,840)
for i in range(1, 13):
    print(i, e1.scalar_multiple(e1.p1, i))

--------------------------------------------------
y^2 = x^3 + 1009x^2 - 72240x + 705600

discriminant: 2982448594944000

Torsion points: 
(56, 0) 位数: 2
(0, 840) 位数: 12
(0, -840) 位数: 12
(120, 2880) 位数: 3
(120, -2880) 位数: 3
(-168, 6048) 位数: 4
(-168, -6048) 位数: 4
(-840, 13440) 位数: 12
(-840, -13440) 位数: 12
(840, 35280) 位数: 6
(840, -35280) 位数: 6
--------------------------------------------------
1 (0, 840)
2 (840.0, 35280.0)
3 (-168.0, 6048.0)
4 (120.0, 2880.0)
5 (-840.0, 13440.0)
6 (56.0, -0.0)
7 (-840.0, -13440.0)
8 (120.0, -2880.0)
9 (-168.0, -6048.0)
10 (840.0, -35280.0)
11 (0.0, -840.0)
12 (inf, inf)
